In [1]:
# run against prodcution 2025-04-09
import pandas as pd

from csae_pyutils import gsheet_to_df
from django.core.exceptions import ObjectDoesNotExist
from tqdm import tqdm
from normdata.utils import import_from_normdata

In [2]:
df = gsheet_to_df("1HwiAHSiBDb2eNfm56Ie9GTNurBIStoqnUxFwszChDmU")
works_df = gsheet_to_df("1aygcH8NTUuDVGSJVIVj1fyI4ATPv0x1Bt8vSLb76z3k")
len(df), len(works_df)

200
200


(2668, 6995)

In [3]:
unique_authors = works_df['authorId'].unique().tolist()
len(unique_authors)
type(unique_authors[10])

float

In [4]:
col, _ = Collection.objects.get_or_create(name="Die Schaubühne")
domain = "schaubuehne"
col_type, _ = CollectionType.objects.get_or_create(name="Projekt")
col.description = """
„Die Schaubühne. Herausgeber: Siegfried Jacobsohn, Berlin 1905–1918“. Digitale Edition. Hrsg. v. Imelda Rohrbacher. Wien: Österreichische Akademie der Wissenschaften (ÖAW) 2024, <a href="https://schaubuehne.oeaw.ac.at">https://schaubuehne.oeaw.ac.at</a>
"""
col.collection_type = col_type
col.save()

In [5]:
broken_wikidata = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    item_id = row["id"]
    name = row["surname"]
    first_name = row["forename"]
    domain_uri = f"https://schaubuehne.oeaw.ac.at/register.html?author={item_id}"
    if isinstance(row["wikidataId"], str):
        if float(item_id) in unique_authors:
            norm_uri = f'http://www.wikidata.org/entity/{row["wikidataId"]}'
            entity = import_from_normdata(norm_uri, "person")
            if entity:
                pmb_uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
                pmb_uri.entity = entity
                pmb_uri.save()
                entity.collection.add(col)
            else:
                broken_wikidata.append(norm_uri)
    elif float(item_id) in unique_authors and isinstance(name, str):
        try:
            uri = Uri.objects.get(uri=domain_uri)
            entity = uri.entity
            entity = Person.objects.get(id=entity.id)
        except ObjectDoesNotExist:
            start_date_written = str(row["birthYear"])
            if start_date_written == "0":
                start_date_written = ""
            end_date_written = str(row["deathYear"])
            if end_date_written == "0":
                end_date_written = ""
            if isinstance(row["forename"], str):
                first_name = row["forename"]
            else:
                first_name = ""
            item = {
                "name": name,
                "first_name": first_name,
                "start_date_written": start_date_written,
                "end_date_written": end_date_written,
                "notes": row["note"]
            }
            entity = Person.objects.create(**item)
            entity.collection.add(col)
            uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
            uri.entity = entity
            uri.save()

          
                
            
        
        
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2668/2668 [01:50<00:00, 24.13it/s]
